In [1]:
from skl2onnx import to_onnx
from sklearn.ensemble import RandomForestClassifier
import sqlalchemy as db
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from skl2onnx.common.data_types import FloatTensorType, Int16TensorType, DoubleTensorType
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report  

from mclLib.featurenames import *
from mclLib.modelpostfix import *
from mclLib.cleaner import *
from mclLib.server_info import *
from mclLib.tester import *

In [2]:

engine = create_engine(mysql_server_uri)
conn = engine.connect()


In [3]:
''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

In [22]:
''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) & (br_full_data['dTradeTime'] >= '2023-03-01') 
br = br_full_data[get_filter]

''' get features name'''
feature_names =  f_name_102
feature_size = len(feature_names)

model_name = 'RF_pow10_multi_v1'

''' set X data '''
X = br[feature_names].to_numpy(dtype=np.float64)

''' set y data '''

bins = [-100, 0.01, 0.04, 100]
labels = [3, 2, 1]
br['label'] = pd.cut(br['fMaxPowerAfterBuyWhile10'], bins=bins, labels=labels)
y = br['label'].to_numpy()

C:\Users\MJ\AppData\Local\Temp\ipykernel_8968\3188624452.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  br['label'] = pd.cut(br['fMaxPowerAfterBuyWhile10'], bins=bins, labels=labels)


In [20]:
from collections import Counter
Counter(y)

Counter({3: 73285, 1: 19505, 2: 35555})

In [160]:
# smote = SMOTE(sampling_strategy=0.85)
# X, y = smote.fit_resample(X, y)

In [21]:
rf = RandomForestClassifier(n_estimators=200, max_depth=12)
rf.fit(X, y)

RandomForestClassifier(max_depth=12, n_estimators=200)

In [156]:
onx = to_onnx(model=rf, options={'zipmap': False},
              initial_types=[('input', DoubleTensorType([None, 102]))],
              final_types=[('output', DoubleTensorType([None])),
                           ('prob', DoubleTensorType([None, 2]))],
              target_opset=17)

with open(onnx_path + model_name + onnx_, "wb") as f:
    f.write(onx.SerializeToString())

In [157]:
cleanAll()

In [23]:
y_pred = rf.predict(X)

In [24]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           1       0.39      0.15      0.21      9908
           2       0.41      0.20      0.26     21823
           3       0.68      0.91      0.78     50446

    accuracy                           0.63     82177
   macro avg       0.49      0.42      0.42     82177
weighted avg       0.57      0.63      0.57     82177



In [172]:
testClassification(y, [y_pred])

총량 :  78290
0 :  10133 , 비율 :  12.942904585515391 (%)
1 :  68157 , 비율 :  87.0570954144846 (%)

============ predict 0 =============
총 횟수 :  381 ,  타겟기준 :  0.0
실제 0 :  151
실제 1 :  230
정답비율 :  39.63254593175853 (%)

============ predict 1 =============
총 횟수 :  77909 , 타겟기준 :  1.0
실제 1 :  67927
실제 0 :  9982
정답비율 :  87.18761632160597 (%)

